In [1]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import numpy as np
from tqdm import tqdm

In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
\

Total memory: 50962169856
Free memory: 1619460096
Used memory: 49342709760


In [4]:
print("Get memory info", torch.cuda.mem_get_info(device=None)) 
print("Get number of devices available: ", torch.cuda.device_count())

print("Memory stats about which device is free: ")

nvidia_smi.nvmlInit()

deviceCount = nvidia_smi.nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    print("Device {}: {}, Memory : ({:.2f}% free): {}(total), {} (free), {} (used)".format(i, nvidia_smi.nvmlDeviceGetName(handle), 100*info.free/info.total, info.total, info.free, info.used))

nvidia_smi.nvmlShutdown()

print("Current Device:  ", torch.cuda.current_device()) 


Get memory info (444006400, 50962169856)
Get number of devices available:  4
Memory stats about which device is free: 
Device 0: b'Quadro RTX 8000', Memory : (0.87% free): 50962169856(total), 444006400 (free), 50518163456 (used)
Device 1: b'Quadro RTX 8000', Memory : (6.16% free): 50962169856(total), 3138846720 (free), 47823323136 (used)
Device 2: b'Quadro RTX 8000', Memory : (57.40% free): 50962169856(total), 29251534848 (free), 21710635008 (used)
Device 3: b'Quadro RTX 8000', Memory : (99.99% free): 50962169856(total), 50958106624 (free), 4063232 (used)
Current Device:   0


In [5]:
torch.cuda.set_device(3)
torch.cuda.empty_cache() 


In [6]:

MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [7]:
class LogicalFallacy(Dataset):
    def __init__(self, dataset, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataset
        self.text = dataset.cleaner_prompt
        self.targets = dataset.label
        self.max_len = max_len
       
    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        

        return {
            'sentence': text,
            
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [8]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
    
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output



In [9]:

def train_loop( train_loader, test_loader, label, test_data, epochs=5):
  train_loss = []
  test_loss = []
  train_accuracy = []
  test_accuracy = []
  model = RobertaClass()
  model.to(device)
  loss_function = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

  test_answers = [[[],[]], [[],[]]]
  for epoch in range(epochs):
    for phase in ['Train', 'Test']:
      if(phase == 'Train'):
        model.train()
        loader = train_loader
      else:
        model.eval()
        loader = test_loader  
      epoch_loss = 0
      epoch_acc = 0
      len(loader)
      for steps, data in tqdm(enumerate(loader, 0)):
        sentence = data['sentence']
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
      
        outputs = model.forward(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)        
        
        epoch_loss += loss.detach()
        _, max_indices = torch.max(outputs.data, dim=1)
        bath_acc = (max_indices==targets).sum().item()/targets.size(0)
        epoch_acc += bath_acc

        if (phase == 'Train'):
          train_loss.append(loss.detach()) 
          train_accuracy.append(bath_acc)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
        else:
          test_loss.append(loss.detach()) 
          test_accuracy.append(bath_acc)
         

      print(f"{phase} Loss: {epoch_loss/steps}")
      print(f"{phase} Accuracy: {epoch_acc/steps}")
  
  torch.save(model, '../models/broad_classifiers/broad_classifier_trained_roberta_prompt_balanced.pt')
  return (train_loss, test_loss, train_accuracy, test_accuracy, test_answers)


In [10]:
def driver_code():
    train_df = pd.read_csv('../data/broad_classifier/updated_edu_train_balanced.csv')
    test_df = pd.read_csv('../data/broad_classifier/updated_edu_dev_balanced.csv')
    

    train_set = LogicalFallacy(train_df, tokenizer, MAX_LEN)
    test_set = LogicalFallacy(test_df, tokenizer, MAX_LEN)

    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
    train_loader = DataLoader(train_set, **train_params)
    test_loader = DataLoader(test_set, **test_params)

    
    value = train_loop( train_loader, test_loader, 'xxxx',  test_df['updated_label'], epochs=5)
    return value
    

In [11]:
vals = driver_code()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer 

Train Loss: 0.6746110320091248
Train Accuracy: 0.6579189686924494


205it [00:05, 38.96it/s]
1it [00:00,  7.14it/s]

Test Loss: 1.0032083988189697
Test Accuracy: 0.6915849673202615


544it [01:19,  6.86it/s]
5it [00:00, 41.20it/s]

...

Test Loss: 1.661028504371643
Test Accuracy: 0.6699346405228758


544it [01:19,  6.84it/s]
5it [00:00, 40.11it/s]

Train Loss: 0.033795859664678574
Train Accuracy: 0.9912523020257827


205it [00:05, 39.18it/s]


Test Loss: 1.7121220827102661
Test Accuracy: 0.7005718954248366
